In [1]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model_name = "quwsarohi/NanoAgent-135M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


def inference(messages, max_new_tokens=256, temperature=0.3, min_p=0.15, **kwargs):
    input_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        min_p=0.15,
        temperature=temperature,
        **kwargs
    )
    return tokenizer.decode(outputs[0][inputs.shape[1] :], skip_special_tokens=True)

In [3]:
messages = [{"role": "user", "content": "Hi! Do you have a name?"}]
print(inference(messages))

I'm sorry for any confusion, but as an AI, I don't have a name. I'm designed to provide information and answer questions about the system. I'm here to help with any queries you might have related to the system.



In [4]:
TOOLS = [
    {
        "name": "web_search",
        "description": "Performs a web search for a query and returns a string of the top search results formatted as markdown with titles, links, and descriptions.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The keyword-rich detailed search query to perform.",
                }
            },
            "required": ["query"],
        },
    },
]

TOOL_TEMPLATE = """You are a helpful AI assistant. You have a set of possible functions/tools inside <tools></tools> tags. 
Based on question, you may need to make one or more function/tool calls to answer user.

You have access to the following tools/functions:
<tools>{tools}</tools>

For each function call, return a JSON list object with function name and arguments within <tool_call></tool_call> tags."""


messages = [
    {"role": "system", "content": TOOL_TEMPLATE.format(tools=json.dumps(TOOLS))},
    {"role": "user", "content": "Can you book a flight?"},
]

print(inference(messages))

I'm sorry, but I don't have the capability to book flights. My current function allows me to perform web searches for a query. If you need help with that, feel free to ask!



In [5]:
def wiki_search(inp: str):
    import wikipedia
    result = None
    try:
        result = wikipedia.summary(inp)
    except wikipedia.exceptions.DisambiguationError as e:
        options = e.options
        results = []
        for option in options:
            results.append(f"# Topic: {option}\n{wikipedia.summary(option)}")
        result = "\n".join(results)
    return json.dumps({"search_result": result.strip()})


def extract_toolcall(input_str):
    import re

    calls = re.findall(r"<tool_call>(.*?)</tool_call>", input_str, re.DOTALL)
    if calls:
        calls = calls[-1].strip()
        try:
            return json.loads(calls)
        except:
            return []
    return []


messages = [
    {"role": "system", "content": TOOL_TEMPLATE.format(tools=json.dumps(TOOLS))},
    {
        "role": "user",
        "content": "What is the scientific name of yellow toadflax?",
    },
]

print("User:", messages[-1]["content"])
llm_tool_call = inference(messages)
print("LLM:", llm_tool_call)

search_query = extract_toolcall(llm_tool_call)[0]["arguments"]["query"]
search_result = wiki_search(search_query)
tool_reply = f"<tool_response>{search_result}</tool_response>"

print("--- Web Result ---")
print(search_result)
print("-------------------\n")

messages += [
    {"role": "assistant", "content": llm_tool_call},
    {"role": "user", "content": tool_reply},
]

print("LLM:", inference(messages))

User: What is the scientific name of yellow toadflax?
LLM: <tool_call>[{"name": "web_search", "arguments": {"query": "yellow toadflax"}}]</tool_call>

--- Web Result ---
{"search_result": "Linaria vulgaris, the common toadflax, yellow toadflax or butter-and-eggs, is a species of flowering plant in the family Plantaginaceae, native to Europe, Siberia and Central Asia. It has also been introduced and is now common in North America."}
-------------------

LLM: The scientific name of yellow toadflax is Linaria vulgaris. It is a species of flowering plant in the family Plantaginaceae, native to Europe, Siberia and Central Asia. It has also been introduced and is now common in North America.

